In [1]:
#import mysql.connector
from sqlalchemy import create_engine
import pymysql
import numpy as np
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader

In [2]:
# credenciales
DB_USER='root'
DB_PASSWORD='chilote64'
DB_NAME=r='recommend'  # test schema with toy data

In [3]:
# IMPLEMENT DATABASE URL
SQLALCHEMY_DATABASE_URI ='mysql+pymysql://{}:{}@localhost:3306/{}'.format(DB_USER,DB_PASSWORD,DB_NAME)

In [4]:
# conneccion
cnx = create_engine(SQLALCHEMY_DATABASE_URI)

In [5]:
qry="select	* from ratings"
df_input=pd.DataFrame(cnx.execute(qry).fetchall())
df_input.columns=['item_id','user_id','rating']

In [6]:
# Preparing the data
df_input=df_input[['user_id','item_id','rating']]
df_input.head()

,user_id,item_id,rating
0,314,1,5
1,439,1,3
2,588,1,5
3,1169,1,4
4,1185,1,4


In [7]:
# With reader class 
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_input[['user_id', 'item_id', 'rating']], reader)

In [8]:
#trainset = data.build_full_trainset()
testsize=0.25
trainset, testset = train_test_split(data, test_size=testsize)

In [ ]:
# define the model algorithm
algo = SVD()

# fit the model with the trainset
algo.fit(trainset)


In [ ]:
# test an accuracy
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

In [ ]:
type(trainset._raw2inner_id_users)
users_dict=trainset._raw2inner_id_users
items_dict=trainset._raw2inner_id_items


In [ ]:
users_dict

In [ ]:
items_dict

In [ ]:
# factores latentes items
algo.qi

In [ ]:
# dimesion matriz factores latentes items
algo.qi.shape

In [ ]:
#factores latantes usuarios
algo.bu

In [ ]:
# dimension matriz factores latentes usuarios
algo.pu.shape

In [43]:
mat_ratings=np.dot(algo.qi,algo.pu.transpose())

In [44]:
mat_ratings.shape

(10000, 52784)

In [45]:
mat_ratings

array([[-0.40448905,  0.16515022,  0.11612089, ...,  0.08687572,
         0.06522121,  0.02682064],
       [ 0.03573149,  0.14018379,  0.17816094, ..., -0.02949169,
        -0.13076227, -0.05313281],
       [ 0.21315938,  0.15760086,  0.2070977 , ...,  0.09335565,
         0.02098122, -0.16428722],
       ...,
       [ 0.04762445,  0.04428889, -0.13049886, ...,  0.25837583,
        -0.09803221, -0.03160385],
       [ 0.0445562 , -0.19657911, -0.06180419, ..., -0.06569155,
        -0.20429557, -0.04875695],
       [-0.04131805, -0.0135689 , -0.05167643, ..., -0.0792198 ,
        -0.16611066,  0.04686996]])

In [ ]:
# suma constante asociada a mediana global
mat_ratings=mat_ratings+algo.trainset.global_mean

In [ ]:
mat_ratings